## Imports

In [1]:
import datetime
import gc

In [2]:
from sklearn.model_selection import StratifiedKFold

In [3]:
cuda_use_gpus(1)

In [4]:
from keras import backend as K
from keras.models import Sequential
from keras.layers import *
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


## Config

In [5]:
feature_list_id = 'oofp_bradleypallen_mlp'

In [6]:
RANDOM_SEED = 42

In [7]:
np.random.seed(RANDOM_SEED)

## Read Data

In [8]:
embedding_matrix = load(aux_data_folder + 'embedding_weights_fasttext_filtered_no_stopwords.pickle')

In [9]:
X_train_q1 = load(features_data_folder + 'X_train_nn_fasttext_q1_filtered_no_stopwords.pickle')
X_train_q2 = load(features_data_folder + 'X_train_nn_fasttext_q2_filtered_no_stopwords.pickle')

In [10]:
X_test_q1 = load(features_data_folder + 'X_test_nn_fasttext_q1_filtered_no_stopwords.pickle')
X_test_q2 = load(features_data_folder + 'X_test_nn_fasttext_q2_filtered_no_stopwords.pickle')

In [11]:
y_train = load(features_data_folder + 'y_train.pickle')

In [12]:
EMBEDDING_DIM = embedding_matrix.shape[-1]
VOCAB_LENGTH = embedding_matrix.shape[0]
MAX_SEQUENCE_LENGTH = X_train_q1.shape[-1]

In [13]:
print(EMBEDDING_DIM, VOCAB_LENGTH, MAX_SEQUENCE_LENGTH)

300 101442 30


## Train Models & Compute Out-of-Fold Predictions

In [14]:
def create_model(params):
    model_q1 = Sequential()

    model_q1.add(Embedding(
        VOCAB_LENGTH,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False,
    ))

    model_q1.add(TimeDistributed(Dense(
        EMBEDDING_DIM,
        activation='relu',
    )))

    model_q1.add(Lambda(lambda x: K.max(x, axis=1), output_shape=(EMBEDDING_DIM, )))

    model_q2 = Sequential()

    model_q2.add(Embedding(
        VOCAB_LENGTH,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False,
    ))

    model_q2.add(TimeDistributed(Dense(
        EMBEDDING_DIM,
        activation='relu'
    )))

    model_q2.add(Lambda(lambda x: K.max(x, axis=1), output_shape=(EMBEDDING_DIM, )))

    model = Sequential()
    model.add(Merge([model_q1, model_q2], mode='concat'))
    model.add(BatchNormalization())

    model.add(Dense(params['num_dense_1']))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Dense(params['num_dense_2']))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Dense(params['num_dense_3']))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Dense(params['num_dense_4']))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        loss='binary_crossentropy', 
        optimizer='adam', 
        metrics=['accuracy']
    )

    return model

In [15]:
model_checkpoint_path = aux_data_folder + 'fold-checkpoint-' + feature_list_id + '.h5'

In [16]:
NUM_FOLDS = 5
NUM_EPOCHS = 200

In [17]:
kfold = StratifiedKFold(
    n_splits=NUM_FOLDS,
    shuffle=True,
    random_state=RANDOM_SEED
)

In [18]:
y_train_oofp = np.zeros_like(y_train, dtype='float64')

In [19]:
y_test_oofp = np.zeros((len(X_test_q1), NUM_FOLDS))

In [20]:
model_params = {
    'num_dense_1': 500,
    'num_dense_2': 225,
    'num_dense_3': 500,
    'num_dense_4': 100,
}

In [21]:
def predict(model, X_q1, X_q2):
    y1 = model.predict(
        [X_q1, X_q2],
        batch_size=1024,
        verbose=1
    ).reshape(-1)
    
    y2 = model.predict(
        [X_q2, X_q1],
        batch_size=1024,
        verbose=1
    ).reshape(-1)
    
    return (y1 + y2) / 2

In [22]:
%%time

for fold_num, (ix_train, ix_val) in enumerate(kfold.split(X_train_q1, y_train)):
    X_fold_train_q1 = np.vstack([X_train_q1[ix_train], X_train_q2[ix_train]])
    X_fold_train_q2 = np.vstack([X_train_q2[ix_train], X_train_q1[ix_train]])

    X_fold_val_q1 = np.vstack([X_train_q1[ix_val], X_train_q2[ix_val]])
    X_fold_val_q2 = np.vstack([X_train_q2[ix_val], X_train_q1[ix_val]])

    y_fold_train = np.concatenate([y_train[ix_train], y_train[ix_train]])
    y_fold_val = np.concatenate([y_train[ix_val], y_train[ix_val]])
    
    print()
    print(f'Fitting fold {fold_num + 1} of {kfold.n_splits}')
    print()
    
    model = create_model(model_params)
    model.fit(
        [X_fold_train_q1, X_fold_train_q2], y_fold_train,
        validation_data=([X_fold_val_q1, X_fold_val_q2], y_fold_val),
#         class_weight=keras_get_class_weights(y_fold_val),

        batch_size=64,
        epochs=NUM_EPOCHS,
        verbose=1,
        
        callbacks=[
            EarlyStopping(
                monitor='val_loss',
                min_delta=0.001,
                patience=3,
                verbose=1,
                mode='auto',
            ),
            ModelCheckpoint(
                model_checkpoint_path,
                monitor='val_loss',
                save_best_only=True,
                verbose=2,
            ),
        ],
    )
        
    # Create out-of-fold prediction.
    model.load_weights(model_checkpoint_path)
    
    y_train_oofp[ix_val] = predict(model, X_train_q1[ix_val], X_train_q2[ix_val])
    y_test_oofp[:, fold_num] = predict(model, X_test_q1, X_test_q2)
    
    # Clear GPU memory.
    K.clear_session()
    del X_fold_train_q1
    del X_fold_train_q2
    del X_fold_val_q1
    del X_fold_val_q2
    del model
    gc.collect()


Fitting fold 1 of 5



/home/yuriyguts/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Train on 646862 samples, validate on 161718 samples
Epoch 1/200
646862/646862 [==============================] - 76s - loss: 0.4630 - acc: 0.7728 - val_loss: 0.4135 - val_acc: 0.8014
Epoch 2/200
646862/646862 [==============================] - 77s - loss: 0.3846 - acc: 0.8196 - val_loss: 0.3843 - val_acc: 0.8191
Epoch 3/200
646862/646862 [==============================] - 77s - loss: 0.3414 - acc: 0.8440 - val_loss: 0.3739 - val_acc: 0.8273
Epoch 4/200
646862/646862 [==============================] - 76s - loss: 0.3074 - acc: 0.8629 - val_loss: 0.3651 - val_acc: 0.8333
Epoch 5/200
646862/646862 [==============================] - 77s - loss: 0.2799 - acc: 0.8772 - val_loss: 0.3645 - val_acc: 0.8373
Epoch 6/200
646862/646862 [==============================] - 77s - loss: 0.2552 - acc: 0.8896 - val_loss: 0.3709 - val_acc: 0.8399
Epoch 7/200
646862/646862 [==============================] - 76s - loss: 0.2347 - acc: 0.8992 - val_loss: 0.3729 - val_acc: 0.8389
Epoch 8/200
646862/646862 [====

646864/646864 [==============================] - 76s - loss: 0.2165 - acc: 0.9074 - val_loss: 0.3992 - val_acc: 0.8392
Epoch 9/200
646864/646864 [==============================] - 77s - loss: 0.2001 - acc: 0.9156 - val_loss: 0.4033 - val_acc: 0.8435
Epoch 00008: early stopping
2343936/2345796 [============================>.] - ETA: 0s
Fitting fold 4 of 5

Train on 646866 samples, validate on 161714 samples
Epoch 1/200
646866/646866 [==============================] - 77s - loss: 0.4617 - acc: 0.7737 - val_loss: 0.4086 - val_acc: 0.8033
Epoch 2/200
646866/646866 [==============================] - 76s - loss: 0.3838 - acc: 0.8202 - val_loss: 0.3798 - val_acc: 0.8204
Epoch 3/200
646866/646866 [==============================] - 77s - loss: 0.3412 - acc: 0.8446 - val_loss: 0.3646 - val_acc: 0.8319
Epoch 4/200
646866/646866 [==============================] - 77s - loss: 0.3074 - acc: 0.8625 - val_loss: 0.3628 - val_acc: 0.8331
Epoch 5/200
646866/646866 [==============================] - 76s -

## Save feature names

In [23]:
feature_names = [
    'oofp_bradleypallen_mlp',
]

In [24]:
save_lines(feature_names, features_data_folder + f'X_train_{feature_list_id}.names')

## Save Train features

In [25]:
y_train_oofp = y_train_oofp.reshape((-1, 1))

In [26]:
save(y_train_oofp, features_data_folder + f'X_train_{feature_list_id}.pickle')

## Save Test features

In [27]:
y_test_oofp_mean = np.mean(y_test_oofp, axis=1).reshape((-1, 1))

In [28]:
save(y_test_oofp_mean, features_data_folder + f'X_test_{feature_list_id}.pickle')